# CNN with Keras by TensorFlow using Smoothed Data

# 1.0 Dependencies and Notes

This notebook was built with the libraries imported below and the following versions:

Pandas 2.2.3 <br>
Numpy 2.0.2 <br>
Altair 5.4.1 <br>
sklearn 1.5.0 <br>
Keras 3.6.0 <br>

Different versions of these libraries may affect the functionality of this notebook.

The purpose of this notebook is to create a convolutional neural network to predict remaining useful life of jet engines using data provided by NASA that was then smoothed to reduce noise. The notebook includes definitions to build the model, fit it, and then explore and store the results. 

Results are stored via a CSV file. There is a function for looping through different parameters, and other functions for viewing, exploring, and saving the results. 

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from numpy import array, hstack
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from keras import Input

In [2]:
print(pd.__version__)
print(np.__version__)
print(alt.__version__)
print(sklearn.__version__)
print(keras.__version__)

2.2.3
2.0.2
5.4.1
1.5.0
3.6.0


## 1.1 Load smoothed batched data and define train and test sets 

In [4]:
#link = 'processed_data_pickle_files_no_smoothing/'
link = 'smoothed_batched_data/'

with open(link + 'test_data_batches.pkl', 'rb') as file:
    test = pickle.load(file)
    
with open(link + 'train_data_batches.pkl', 'rb') as file:
    train = pickle.load(file)    
    
with open(link + 'train_target_values.pkl', 'rb') as file:
    y_train = pickle.load(file)
    
with open(link + 'true_rul_values.pkl', 'rb') as file:
    y_test = pickle.load(file)

In [5]:
display(train.shape)
display(y_train.shape)
display(test.shape)
display(y_test.shape)

(17731, 30, 14)

(17731,)

(100, 30, 14)

(100,)

In [6]:
#Initialize so definitions don't throw an error. Will be redefined in subsequent section. 
default_params = {}
loops = {}

# 2.0 Definitions

In [7]:
def make_cnn(params, print_summary = False):
    
    """
    Intializes a convolutional neural network using TensorFlow Keras Library. 
    Parmeters are entered as a library. 
    
    Inputs:
    params: library of input parameters, must include 'sec_conv', '#filters_conv1', 'filter_size_conv1', 
            'act_func_conv1', 'input_1', 'input_2', 'dense_neurons', 'act_func_dense'; may need to include others. 
    print_summary: boolean that dictates whether to print a summary of the constructed model 
            (provides number of nuerons in each network, etc.)
    """
    
    
    #A sequential model can have different NN layers added. Use a list to define layers, or use add method.
    #Sequential models only work for one input tensor and one output tensor (including for each layer).
    model = Sequential()
    
    #Input(shape = (params['input_1'], params["input_2"]))
    
    #Adds a 1-dimensional convolution layer (only moves in one direction: down each 'sample' of timeseries).
    model.add(Conv1D(filters = params["#filters_conv1"], #dimension of output space (number of filters)
                     kernel_size = params["filter_size_conv1"], #size of the convolution window
                     strides = 1, #convolution layer stride length
                     padding = 'valid', #zero padding at ends of convolutions, options: 'valid' (no padding), 'same'
                     activation=params["act_func_conv1"], #activation function to derive non-linear relationships; default is None.
                     input_shape=(params['input_1'], params["input_2"]) #steps followed by number of features (same order as input)
                    )) 
    
    #Adds a second 1D convolution filter if user requests.
    if params["sec_conv"] == True:
        model.add(Conv1D(filters = params["#filters_conv2"], #dimension of output space (number of filters)
                     kernel_size = params["filter_size_conv2"], #size of the convolution window
                     activation=params["act_func_conv2"], #activation function to derive non-linear relationships; default is None.
                        )) 
    
    #Adds a pooling layer to reduce dimensions/computation time - Max or Average is available. 
    #Downsizes the input from the Conv1D layer, so structure is kept but some info is lost.
    model.add(MaxPooling1D(pool_size = 2)) #Number of features considered at once.
    
    #Adds a flattening layer, which makes the 3D data into a 1D array so it's compatible with Dense Layers.
    model.add(Flatten())
    
    #Adds a "dense" layer, which is a regular NN layer. This interprets the output of the previous layers.
    model.add(Dense(params['dense_neurons'], #First parameter is the output space dimensionality.
                    activation = params['act_func_dense'])) 
    
    #Adds a second dense layer to reduce the 50 neurons to a single output.
    #This is done seperately for each batch.
    model.add(Dense(1))
    
    #This method configures the model for training. It's where you choose parameters.
    #Other parameter available: loss_weights, metrics, weighted_metrics, etc. 
    model.compile(optimizer = 'adam', loss = 'mse', metrics = ["mean_squared_error", 
                                                               "root_mean_squared_error", 
                                                               "mean_absolute_error", 
                                                               "mean_absolute_percentage_error", 
                                                               "r2_score"])
    
    if print_summary == True:
        model.summary()
    
    return model

In [8]:
def cnn_run_for_log(model, train_data, train_target, test_data, test_target, params):
    
    """ 
    Trains an input model on the input train data, then collects various scoring metrics of both the 
    train and test data. The input parameters dictionary is then concatenated with the metrics to provide 
    a dictionary of both the metrics and input parameters used. 
    
    Inputs:
    
    Model: CNN model from the previous function above ('make_cnn') or CNN defined via other means
    
    Various data inputs: Train and Test, plus targets
    
    params: library of parameters. Must include 'perform_validation' and '#_epochs'.
    
    Output: library of train and test parameters, along with parameters included in the 'params' input. 
    
    """
    
    if params['perform_validation'] == True:
        v_s = 0.2
    else:
        v_s = None
    
    history = model.fit(train_data, 
                      train_target, 
                      validation_split = v_s, 
                      epochs = params["#_epochs"], 
                      verbose = 0
                     )
    logger = {}
    
    y_hat = model.predict(test_data, verbose = 0)
    
    logger["train_MSE"] = history.history["mean_squared_error"][-1]
    logger["test_MSE"] = mean_squared_error(test_target, y_hat).item()
    
    logger["train_RMSE"] = history.history["root_mean_squared_error"][-1]
    logger["test_RMSE"] = np.sqrt(mean_squared_error(test_target, y_hat)).item()
    
    logger["train_MAE"] = history.history["mean_absolute_error"][-1]
    logger["test_MAE"] = mean_absolute_error(test_target, y_hat).item()
    
    logger["train_MAPE"] = history.history["mean_absolute_percentage_error"][-1]
    logger["test_MAPE"] = np.mean(np.abs((test_target - y_hat) / y_test)).item() * 100
    
    logger["train_R2"] = history.history["r2_score"][-1]
    logger["test_R2"] = r2_score(test_target, y_hat)
    
    logger.update(params)
    
    return logger
    
    

In [9]:
def add_to_logger(new_instance, existing_dict):
    
    """
    Takes in a new instance of the function 'run_cnn_for_log' which returns the performance metrics 
    for a CNN using the listed input parameters. It then adds that instance to a dictionary of lists,
    where each index in the list represents a new run of 'run_cnn_for_log'. This is intended to be used
    in running loops during parameter tuning to keep track of which parameters perform the best. 
    
    Input:
    new_instance: a dictionary of the most recent parameters and performance metrics
    existing_dict: a dictionary of lists that keep a record of performance metrics and the parameters that
            led to those results
    
    Output:
    An updated record of parameters/performance metrics in which the most recent parameters are added to the record
    
    """
    
    if existing_dict == None:
        record = {}
        for key in new_instance.keys():
            record[key] = []
    else:
        record = existing_dict.copy()
        
    for key in record.keys():
        l = record[key]
        if key in new_instance.keys():
            l.append(new_instance[key])     
        else:
            l.append(np.nan)
        record[key] = l
        
    return record

In [10]:
def loop_through_parameters(loops, 
                            parameters, 
                            train = train, 
                            y_train = y_train, 
                            test = test, 
                            y_test = y_test):
    """
    Runs parameter loops for model and records the resulting metrics. Returns a dictionary that is 
    a log of the results, where each key represents a parameter or performance metric, and each item is
    a list where the indexes represent the runs in chronological order. 
    
    Inputs:
    loops: a dictionary of the the keys and values to loop through.
    params: a dictionary with all the parameters neccessary to build the CNN, train it, and acquire the 
    results using the functions defined previously.
    
    Output:
    A dictionary of lists with input parameters and resultant performance metrics. Can easily be used
    to construct a dataframe. 
    """
    
    record = None
    keys = list(loops.keys())
    
    def nested_function(p1 = None, p2 = None, p3 = None, p4 = None):
        parameters[keys[0]] = p1
        if len(keys) > 1:
            parameters[keys[1]] = p2
        if len(keys) > 2:
            parameters[keys[2]] = p3
        if len(keys) > 3:
            parameters[keys[3]] = p4
            
        #print(parameters)
        model = make_cnn(parameters)
        new_instance = cnn_run_for_log(model, train, y_train, test, y_test, parameters)
        r = add_to_logger(new_instance, record)
        return r
        
    if len(loops.keys()) == 1:
        for p_1 in loops[keys[0]]:
            record = nested_function(p1 = p_1)
            
    elif len(loops.keys()) == 2:
        for p_1 in loops[keys[0]]:
            for p_2 in loops[keys[1]]:
                record = nested_function(p1 = p_1, p2 = p_2)
                
    elif len(loops.keys()) == 3:
        for p_1 in loops[keys[0]]:
            for p_2 in loops[keys[1]]:
                for p_3 in loops[keys[2]]:
                    record = nested_function(p1 = p_1, p2 = p_2, p3 = p_3)
                    
    else:
        for p_1 in loops[keys[0]]:
            for p_2 in loops[keys[1]]:
                for p_3 in loops[keys[2]]:
                    for p_4 in loops[keys[3]]:
                        record = nested_function(p1 = p_1, p2 = p_2, p3 = p_3, p4 = p_4)
                        
    return record
        

In [11]:
def add_to_cnn_log(record, link = 'CNN_log.csv', save_changes = False):
    
    """
    Combines most recent group of models with those saved in the log file. Includes designating a
    tuning group based on the most recent tuning group in the log file.
    
    Inputs:
    record: most recent record set of parameter tunings, as returned by above functions.
    link: pathway and filename for the saved log file, it if exists. If it doesn't exist, this 
        function won't work.
    save_changes: designates whether to save the updates to the CSV file that stores the model parameter tuning results.
    
    Output:
    A dataframe of the combines records on file and the most recent turning group. 
    """
    
    df = pd.read_csv(link)
    group_num = df['tuning_group'].max() + 1
    r = pd.DataFrame(record)
    r.insert(0, 'tuning_group', group_num)
    
    df = pd.concat([df, r], ignore_index = True)

    if save_changes == True:
        df.to_csv(link, index = False)
    
    return df

In [12]:
def plot_results(data, fields):
    
    """
    Makes a simple Altair Chart for compairing results visually.
    
    Input:
    data: Dataframe that includes the fields from the most current record or from the CNN_log saved as a CSV.
    fields: designated fields to encode using shape and color. Default are the first two designated in the 
        most recent record.
        
    Output:
    A chart comparing changes in the designated fields, mapped against test RMSE and difference between
        train and test RMSE.
    """

    data['RMSE_diff'] = data['test_RMSE'] - data['train_RMSE']
    fields_to_keep = ["test_RMSE", "train_RMSE", "RMSE_diff"] + fields
    data = data[fields_to_keep]

    chart = alt.Chart(data).mark_point().encode(x = alt.X("test_RMSE").scale(zero = False), 
                                                y = 'RMSE_diff', 
                                                color = fields[0] + ":N", 
                                                shape = fields[1] + ":N")

    return chart

def show_df_of_results(data, fields):
    data['RMSE_diff'] = data['test_RMSE'] - data['train_RMSE']
    fields_to_keep = ["test_RMSE", "train_RMSE", "RMSE_diff"] + fields
    data = data[fields_to_keep]
    return data

# 3.0 Building Models and Exploring Results

## Define the default parameters and those to change while looping. 

The cell below was used while tuning the CNN model. Different "tuning groups" were used in loops and then investigated by studying plots and dataframes. Thereafter, a new tuning group would be created for further investigation. The goal was to find the best model that maximized the accuracy but minimized overtraining. 

For repeatability, the functions called out in the blocks above were used in the tuning. 

In [13]:
#Parameter List
default_params = {"sec_conv": True, 
          "perform_validation": False,
          "num_train_samples": train.shape[0],
          "input_1": train.shape[1], 
          "input_2": train.shape[2],
          "#_epochs": 4,
          "#filters_conv1": 64,
          "filter_size_conv1": 3,
          "act_func_conv1": 'sigmoid',
          "#filters_conv2": 384,
          "filter_size_conv2": 5,
          "act_func_conv2": 'relu',
          "dense_neurons": 50,
          "act_func_dense": 'relu',
         }

loops = {
         "#_epochs": [2, 3, 4, 5, 6, 9, 15], #GROUP 1 - USED 4 IN GROUP 2
         "#filters_conv1": [32, 64, 128, 256, 512], #GROUP 1 - USED 48 IN GROUP 2
         "filter_size_conv1": [2, 3, 4, 5, 6, 8], #GROUP 1
#          "act_func_conv1": ['sigmoid', 'tanh', 'relu'], #GROUP 2 - KEEP AS SIGMOID
#          "#filters_conv1": [16, 32, 48, 64, 80, 96, 112, 128], #GROUP 3 - go with 48 and add second conv layer
#          "#filters_conv2": [16, 32, 48, 64, 80, 96, 128, 256, 384, 512], #GROUP 4 - KEEP 512
#          "act_func_conv2": ['relu', 'sigmoid', 'tanh'], #GROUP 4 - KEEP RELU
#          "filter_size_conv2": [2, 3, 4, 5, 6, 8, 10], #GROUP #5
#          "perform_validation": [True, False], #GROUP #5
#          "#_epochs": [3, 4, 5, 6, 7], #GROUP 6
#          "#filters_conv1": [16, 32, 64, 96, 128], #GROUP 6
#          "filter_size_conv1": [2, 3, 4, 5], #GROUP 6
#          "#filters_conv1": [48, 56, 64, 72], #GROUP 7
#          "#filters_conv2": [48, 96, 256, 384, 512], #GROUP 7
#          "filter_size_conv2": [3, 5, 8], #GROUP 7
#          "#filters_conv1": [32, 32, 32, 64, 64, 64], #GROUP 8
#          "#filters_conv2": [48, 48, 48, 96, 96, 96, 384, 384, 384], #GROUP 8  
#            "dense_neurons": [20, 40, 60, 80, 100, 200, 300, 400, 500], #GROUP 9
#            "act_func_dense": ['relu', 'sigmoid', 'tanh'], #GROUP 9
#            "dense_neurons": [80, 150, 180, 200, 220, 250, 300], #GROUP 10
#            "perform_validation": [True, False], #GROUP #10
        }

The loops are ran with in the block below using the inputs designated in the block above. 

In [14]:
#for each key and associated list in 'loops', make a record of results for different parameters.
record = loop_through_parameters(loops, default_params)

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sbmoi\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_reg

## Compare results graphically and in a dataframe

Results from the most recent tuning group were viewed below. 

In [15]:
print(loops.keys())
plot_results(pd.DataFrame(record), list(loops.keys()))

# display(plot_results(pd.DataFrame(record), ['#filters_conv1', '#filters_conv2']))
# display(plot_results(pd.DataFrame(record), ['#filters_conv2', 'filter_size_conv2']))

dict_keys(['#_epochs', '#filters_conv1', 'filter_size_conv1'])


alt.Chart(...)

A table of the most recent results was viewed below, sorted by test RMSE or train/test performance difference. 

In [16]:
df = show_df_of_results(pd.DataFrame(record), list(loops.keys()))
df.sort_values("RMSE_diff")
df.sort_values("test_RMSE")

,test_RMSE,train_RMSE,RMSE_diff,#_epochs,#filters_conv1,filter_size_conv1
42,25.262480,10.654352,14.608128,3,128,2
7,26.549247,12.119228,14.430019,2,64,3
174,26.637015,10.111709,16.525306,9,512,2
13,26.834564,11.369565,15.464999,2,128,3
21,27.031874,12.796519,14.235355,2,256,5
...,...,...,...,...,...,...
185,34.730332,8.011386,26.718947,15,32,8
115,34.770065,9.961284,24.808782,5,512,3
56,35.066107,11.017135,24.048973,3,512,4
193,35.522699,9.039252,26.483447,15,128,3


The cell below was used to initialize the performance log CSV. 

In [17]:
#Initialize the log with the first tuning group. 
#FILENAME REMOVED - DO NOT OVERWRITE LOG!

# df = pd.DataFrame(record)

# df.insert(0, "tuning_group", 1)

# df.to_csv("CNN_SMOOTH....csv")

The below block saves the most recent tuning group to the CSV log.  

In [280]:
#add_to_cnn_log(record, save_changes = True)

,Unnamed: 0,tuning_group,train_MSE,test_MSE,train_RMSE,test_RMSE,train_MAE,test_MAE,train_MAPE,test_MAPE,...,input_2,#_epochs,#filters_conv1,filter_size_conv1,act_func_conv1,#filters_conv2,filter_size_conv2,act_func_conv2,dense_neurons,act_func_dense
0,0.0,1,355.895904,337.148566,18.865204,18.361606,14.578653,13.619847,59482784.0,149.622047,...,14,3,32,3,sigmoid,16,5,relu,50,relu
1,1.0,1,261.337524,261.644746,16.165937,16.175436,12.574144,12.334615,64940664.0,137.768392,...,14,3,64,3,sigmoid,16,5,relu,50,relu
2,2.0,1,237.478317,241.570480,15.410332,15.542538,12.074533,11.737319,75410768.0,144.281251,...,14,3,128,3,sigmoid,16,5,relu,50,relu
3,3.0,1,213.664215,233.558094,14.617257,15.282608,11.420168,11.452817,68126056.0,140.974781,...,14,3,256,3,sigmoid,16,5,relu,50,relu
4,4.0,1,204.825729,241.239781,14.311734,15.531896,11.161909,11.784251,52648680.0,151.277022,...,14,3,512,3,sigmoid,16,5,relu,50,relu
5,5.0,1,287.872467,258.666353,16.966804,16.083108,13.105846,11.970575,59131460.0,147.409903,...,14,4,32,3,sigmoid,16,5,relu,50,relu
6,6.0,1,210.422592,227.718637,14.505950,15.090349,11.263729,11.352382,38350452.0,145.565197,...,14,4,64,3,sigmoid,16,5,relu,50,relu
7,7.0,1,203.550232,246.400471,14.267103,15.697148,11.199593,12.066667,56108536.0,151.755213,...,14,4,128,3,sigmoid,16,5,relu,50,relu
8,8.0,1,183.459992,224.885678,13.544741,14.996189,10.270009,11.029661,42406780.0,147.017653,...,14,4,256,3,sigmoid,16,5,relu,50,relu
9,9.0,1,193.699326,254.986108,13.917590,15.968284,10.732617,12.037212,41657932.0,137.457930,...,14,4,512,3,sigmoid,16,5,relu,50,relu


The below block is used to explore the entire CSV log to compare results. Also organized by performace/overfitting to find the best parameters to test. 

In [12]:
# df = pd.DataFrame(record)
# print(df.columns)
df = pd.read_csv("CNN_log.csv")

##USE CTRL + "/" TO COMMENT OUT FIELDS
df = df[[
    'tuning_group',
#     'train_MSE', 
#     'test_MSE', 
    'train_RMSE', 
    'test_RMSE', 
#     'train_MAE',
#     'test_MAE', 
#     'train_MAPE', 
#     'test_MAPE', 
#     'train_R2', 
#     'test_R2',
    'sec_conv', 
    'perform_validation', 
#     'num_train_samples', 
#     'input_1',
#     'input_2', 
    '#_epochs', 
    '#filters_conv1', 
    'filter_size_conv1',
    'act_func_conv1', 
    '#filters_conv2', 
    'filter_size_conv2',
    'act_func_conv2',
    'dense_neurons',
    'act_func_dense',
    ]]
df.insert(3, "RMSE_diff", df["test_RMSE"] - df["train_RMSE"])
pd.set_option('display.max_rows', None)
df = df.sort_values("test_RMSE").reset_index()[:20]
df

,index,tuning_group,train_RMSE,test_RMSE,RMSE_diff,sec_conv,perform_validation,#_epochs,#filters_conv1,filter_size_conv1,act_func_conv1,#filters_conv2,filter_size_conv2,act_func_conv2,dense_neurons,act_func_dense
0,397,10,14.001601,14.220760,0.219159,True,False,4,64,3,sigmoid,384,5,relu,200,relu
1,178,6,14.448936,14.383256,-0.065680,True,False,4,64,3,sigmoid,512,5,relu,50,relu
2,343,8,14.186660,14.438518,0.251858,True,False,4,64,3,sigmoid,384,5,relu,50,relu
3,378,9,14.239456,14.463041,0.223585,True,False,4,64,3,sigmoid,384,5,relu,200,relu
4,361,8,14.448208,14.474606,0.026398,True,False,4,64,3,sigmoid,384,5,relu,50,relu
5,334,8,14.154706,14.494445,0.339739,True,False,4,32,3,sigmoid,384,5,relu,50,relu
6,132,4,14.143855,14.526599,0.382744,True,False,4,48,3,sigmoid,512,5,relu,50,relu
7,113,4,13.828195,14.526994,0.698799,True,False,4,48,3,sigmoid,48,5,tanh,50,relu
8,354,8,14.309043,14.532130,0.223087,True,False,4,64,3,sigmoid,48,5,relu,50,relu
9,335,8,14.088375,14.552896,0.464521,True,False,4,32,3,sigmoid,384,5,relu,50,relu


# 4.0 Final Model

Using the above tuning, a final model was chosen with the parameters below. Then, the model was trained with the same data and saved into a pickle file. 

In [294]:
#Final parameter list
params = {"sec_conv": True, 
          "perform_validation": False,
          "num_train_samples": train.shape[0],
          "input_1": train.shape[1], 
          "input_2": train.shape[2],
          "#_epochs": 4,
          "#filters_conv1": 64,
          "filter_size_conv1": 3,
          "act_func_conv1": 'sigmoid',
          "#filters_conv2": 384,
          "filter_size_conv2": 5,
          "act_func_conv2": 'relu',
          "dense_neurons": 200,
          "act_func_dense": 'relu',
         }

Model is built/defined in block below, and the model structure summary is provided as output. 

In [297]:
cnn = make_cnn(params, print_summary = True)

Model: "sequential_836"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1465 (Conv1D)            │ (None, 28, 64)         │         2,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1466 (Conv1D)            │ (None, 24, 384)        │       123,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_831               │ (None, 12, 384)        │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_831 (Flatten)           │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1662 (Dense)              │ (None, 200)            │       921,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1663 (Dense)              │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,048,017 (4.00 MB)

 Trainable params: 1,048,017 (4.00 MB)

 Non-trainable params: 0 (0.00 B)

Model is trained in the block below, and then saved to a pickle file for future use. 

In [299]:
history = cnn.fit(train, 
                  y_train,  
                  epochs = params["#_epochs"], 
                  verbose = 0,
                    )

with open('CNN_model_trained.pkl', 'wb') as f:
    pickle.dump(history, f)